In [2]:
import numpy as np
A,a,b,B,n,V = 1
np.set_printoptions(suppress=True)


TypeError: cannot unpack non-iterable int object

In [ ]:
# find shape of matrix
A.shape()

# multiply two matrices together
np.dot(a,b)

# find trace (sum of diagonal elements in array starting from top left)
np.trace(n)

# find matrix determinant
np.linalg.det(n)

# finding matrix inverse
np.linalg.inv(n)

# create identity matrix of size n
np.eye(n)

# create a AxB matrix of random numbers drawn from a normal distribution with mean 0 and variance 1
np.random.randn(A,B)

# A 3x3 diagonal matrix with (1, 5, 1) along its principal diagonal
np.diag((1, 5, 1))

# To create a 1x3 matrix of ones
np.ones((1,3))

# find norm of vector V
np.linalg.norm(V)

In [ ]:

# convert to row echelon form
A = np.array([[2, 4, 6], [3, 8, 5], [-1, 1, 2]], dtype='float')
b = np.array([22, 27, 2])
Ab = np.hstack([A, b.reshape(-1, 1)])
n = len(b)

#Converts augmented matrix to row echelon form
for i in range(n):
    ############ MODIFICATION START #############
    #If leading number is zero, swap with a row below where leading number is non-zero
    if Ab[i,i] == 0:
        temp = Ab[i].copy()

        for k in range (i + 1, n):
            if Ab[k, i] != 0:
                Ab[i] = Ab[k].copy()
                Ab[k] = temp.copy()
                break
    ############ MODIFICATION END #############
    a = Ab[i]
    for j in range(i + 1, n):
        b = Ab[j]
        m = b[i] / a[i]
        Ab[j] = b - m * a

print(Ab)

# answer output
x = Ab[:, n]
x

In [ ]:
#Converts FROM row echelon form to reduced row echelon form
for i in range(n - 1, -1, -1):
    Ab[i] = Ab[i] / Ab[i, i]
    a = Ab[i]

    for j in range(i - 1, -1, -1):
        b = Ab[j]
        m = a[i] / b[i]
        Ab[j] = a - m * b

print(Ab)

# answer output
x = Ab[:, n]
x

In [ ]:
import pandas as pd
data = pd.read_csv("Practical3.csv")

In [ ]:
#Find sample mean vector. axis=0 indicates each column in the data matrix is a variable.
x_bar = np.transpose(np.mean(data, axis=0))

#Find sample covariance matrix. rowvar=0 indicates each column in the data matrix is a variable.
C = np.cov(data, rowvar=0)

#Find sample correlation matrix
R = np.corrcoef(data, rowvar=0)

#Find generalised sample variance
np.linalg.det(C)

#Find total sample variance
np.trace(C)

In [2]:
# extract eigenvalues and eigenvectors
eigvals, eigvecs = np.linalg.eig(A)

#Put the eigenvalues into a dataframe
eigvals_df = pd.DataFrame(np.round(eigvals,4) ) # might need to transpose ur eigvals
eigvals_df.index = ['PC{}'.format(i) for i in range(1,len(eigvals_df)+1)]

#Put the eigenvectors into a dataframe
eigvecs_df = pd.DataFrame(np.round(eigvecs,4)) #might need to transpose ur eigvecs
eigvecs_df.index = ['PC{}'.format(i) for i in range(1,len(eigvecs_df)+1)]

# Its corresponding eigenvector is in column 1 of eigvecs
print("First eigenvalue is: ", eigvals[0])# subject to rounding errors
print("Corresponding eigenvector is: ", eigvecs[:,0])
print("Its magnitude is: ", np.linalg.norm(eigvecs[:,0])) # subject to rounding error

# Its corresponding eigenvector is in column 1 of eigvecs
print("First eigenvalue is: ", eigvals[0])# subject to rounding errors
print("Corresponding eigenvector is: ", eigvecs[:,0])
print("Its magnitude is: ", np.linalg.norm(eigvecs[:,0])) # subject to rounding error

NameError: name 'np' is not defined

In [ ]:
# implementing PCA using sklearn

from sklearn.decomposition import PCA
pca = PCA(n_components='fill_in').fit(data)
pca_samples = pca.transform(data)

# Create dataframe for centred scores
scores_centred = pd.DataFrame(pca_samples, columns = ['PC1', 'PC2'])

In [ ]:
def pca_results(data, pca):

    # Dimension indexing
    dimensions = ['PC {}'.format(i) for i in range(1,len(pca.components_)+1)]

    # PCA components
    components = pd.DataFrame(np.round(pca.components_, 4), columns = data.keys())
    components.index = dimensions

    #PCA eigenvalues
    ev = pca.explained_variance_.reshape(len(pca.components_), 1)
    eigenvalues = pd.DataFrame(np.round(ev, 4), columns = ['Eigenvalue'])
    eigenvalues.index = dimensions

    # PCA explained variance
    ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
    variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
    variance_ratios.index = dimensions

    # PCA cum explained variance
    cum_ratios = pca.explained_variance_ratio_.cumsum().reshape(len(pca.components_), 1)
    cum_variance_ratios = pd.DataFrame(np.round(cum_ratios, 4), columns = ['Cumulative Explained Variance'])
    cum_variance_ratios.index = dimensions
    # Return a concatenated DataFrame
    return pd.concat([eigenvalues, variance_ratios, cum_variance_ratios, components], axis = 1)

pca_results = pca_results(data, pca)
pca_results

In [ ]:
# This function plots the biplot.
# Pass centred scores dataframe and returns of PCA to this function. Optional width, height and margin
# This function returns the axes of the loading plot
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


def biplot(scores, pca, width=5, height=5, margin=0.5):
    plt.figure(figsize=(width, height))
    ax = sns.scatterplot(x='PC1', y='PC2', data = scores)

    #Set margins and limits for figure
    x_min = min(scores['PC1'].min(), pca.components_[0,:].min(), 0) - margin
    x_max = max(scores['PC1'].max(), pca.components_[0,:].max(), 0) + margin
    y_min = min(scores['PC2'].min(), pca.components_[1,:].min(), 0) - margin
    y_max = max(scores['PC2'].max(), pca.components_[1,:].max(), 0) + margin

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_title("Biplot", fontsize=14)

    #Scaling factor for text position
    text_pos = 0.2

    for i, v in enumerate(pca.components_.T):
        ax.arrow(0, 0, v[0], v[1], head_width=0.1, head_length=0.1, linewidth=2, color='red')
        ax.text(v[0], v[1]+text_pos, data.columns[i], color='black', ha='center', va='center', fontsize=12)

    plt.plot([x_min, x_max], [0, 0], color='k', linestyle='--', linewidth=1)
    plt.plot([0, 0], [y_min, y_max], color='k', linestyle='--', linewidth=1)
    return ax

plt.clf()
ax6 = biplot(scores_centred, pca, width=10, height=5, margin=0.3)
plt.show()

In [ ]:

def scoreplot(data, scores, pca, width=5, height=5, margin=0.5):
    plt.figure(figsize=(width, height))
    ax = sns.scatterplot(x="PC1", y="PC2", data = scores)
    text_pos = 0.15
    # Print text
    for i, v in enumerate(np.array(scores)):
        ax.text(v[0], v[1]+text_pos, data.iloc[i,0], color='black', ha='center', va='center', fontsize=12)

    #Set limits for figure
    x_min = min(scores['PC1'].min(),0)-margin
    x_max = max(scores['PC1'].max(),0)+margin
    y_min = min(scores['PC2'].min(),0)-margin
    y_max = max(scores['PC2'].max(),0)+margin

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_title("Score plot", fontsize=14)
    plt.plot([x_min, x_max], [0, 0], color='k', linestyle='--', linewidth=1)
    plt.plot([0, 0], [y_min, y_max], color='k', linestyle='--', linewidth=1)
    return ax

plt.clf()
ax4 = scoreplot(data, scores_centred, pca)
plt.show()

In [ ]:

def loadingplot(data, pca, width=5, height=5, margin=0.5):

    fig, ax = plt.subplots(figsize = (width,height))

    #Set limits for figure
    x_min = min(pca.components_[0,:].min(),0)-margin
    x_max = max(pca.components_[0,:].max(),0)+margin
    y_min = min(pca.components_[1,:].min(),0)-margin
    y_max = max(pca.components_[1,:].max(),0)+margin

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    #Scaling factor for text position
    text_pos = 0.2

    for i, v in enumerate(pca.components_.T):
        ax.arrow(0, 0, v[0], v[1], head_width=0.1, head_length=0.1, linewidth=2, color='red')
        ax.text(v[0], v[1]+text_pos, data.columns[i], color='black', ha='center', va='center', fontsize=12)

    plt.plot([x_min, x_max], [0, 0], color='k', linestyle='--', linewidth=1)
    plt.plot([0, 0], [y_min, y_max], color='k', linestyle='--', linewidth=1)
    ax.set_xlabel("PC1", fontsize=14)
    ax.set_ylabel("PC2", fontsize=14)
    ax.set_title("Loading plot", fontsize = 14)

    return ax

plt.clf()
ax5 = loadingplot(data, pca, width=5, height=7)
plt.show()

In [ ]:
#Scree plot
import matplotlib.ticker as ticker
plt.clf()
fig1, ax1 = plt.subplots()
ax1.plot(np.arange(1, len(pca.explained_variance_)+1), pca.explained_variance_)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax1.set(xlabel = 'Component number', ylabel = 'Eigenvalue', title = 'Screeplot')
plt.show()

# Correlation

In [ ]:
# scaling the data before PCA
from sklearn.preprocessing import scale
data_reduced = pd.DataFrame(scale(data), columns=['Bread','Vegetables','Fruit','Meat','Poultry','Milk','Wine'])
data_reduced.head()